In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
cd drive/MyDrive/Stock_Prediction

[Errno 2] No such file or directory: 'drive/MyDrive/Stock_Prediction'
/content/drive/.shortcut-targets-by-id/1ghHRBDBpcK1VoZtVrE4wNbQ-8HihIIEZ/Stock_Prediction


In [22]:
!pip3 install nselib

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import numpy as np
from tqdm import tqdm
from nselib import capital_market

In [24]:
#
#
#
#.                 NO NEED TO RUN THIS CELL, UNLESS YOU WANT TO REFRESH THE LIST OF 1000 STOCKS
#
#
#

'''
def fetch_data(offset):
    url = f'https://finance.yahoo.com/screener/unsaved/8ca05005-bc66-45c2-b731-c2710bb289b2?offset={offset}'

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the stock data
    table = soup.find('table', {'class': 'W(100%)'})
    if table is None:
        print(f"No table found at offset {offset}")
        return None, None

    rows = table.find_all('tr')

    # Extract the headers
    headers = [header.text for header in rows[0].find_all('th')]

    # Extract the rows
    data = []
    for row in rows[1:]:
        cols = row.find_all('td')
        data.append([col.text for col in cols])

    return headers, data

def main():
    all_data = []
    headers = None

    for offset in range(0, 1050, 25):
        print(f"Fetching data from offset {offset}...")
        page_headers, page_data = fetch_data(offset)
        if page_data:
            if headers is None:
                headers = page_headers
            all_data.extend(page_data)

    # Convert to DataFrame and save to CSV
    if headers and all_data:
        df = pd.DataFrame(all_data, columns=headers)
        df.to_excel('top_1000_stocks_nse.xlsx', index=False)
        print("Data extracted and saved successfully")
    else:
        print("No data was fetched")

if __name__ == "__main__":
    main()
'''

'\ndef fetch_data(offset):\n    url = f\'https://finance.yahoo.com/screener/unsaved/8ca05005-bc66-45c2-b731-c2710bb289b2?offset={offset}\'\n\n    headers = {\n        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"\n    }\n\n    response = requests.get(url, headers=headers)\n    soup = BeautifulSoup(response.content, \'html.parser\')\n\n    # Find the table containing the stock data\n    table = soup.find(\'table\', {\'class\': \'W(100%)\'})\n    if table is None:\n        print(f"No table found at offset {offset}")\n        return None, None\n\n    rows = table.find_all(\'tr\')\n\n    # Extract the headers\n    headers = [header.text for header in rows[0].find_all(\'th\')]\n\n    # Extract the rows\n    data = []\n    for row in rows[1:]:\n        cols = row.find_all(\'td\')\n        data.append([col.text for col in cols])\n\n    return headers, data\n\ndef main():\n    all_data = []\n    headers = Non

In [2]:

def get_stock_data(ticker):

    # Calculate the date 2 years ago from today
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*2)

    if ticker[-3:]=='.BO' or ticker[-3:]=='.NS' or ticker[0]=='^':

        # Fetch historical data
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date)
        closing_prices = data[['Close', 'High', 'Low', 'Volume']]


    else:


        # Convert to string in the desired format
        end_date = end_date.strftime("%d-%m-%Y")
        start_date = start_date.strftime("%d-%m-%Y")

        data = capital_market.index_data(ticker, from_date= start_date, to_date= end_date)
        closing_prices = data[['CLOSE_INDEX_VAL', 'HIGH_INDEX_VAL', 'LOW_INDEX_VAL', 'TRADED_QTY']]

        closing_prices = closing_prices.rename(columns={
            'CLOSE_INDEX_VAL': 'Close',
            'HIGH_INDEX_VAL': 'High',
            'LOW_INDEX_VAL': 'Low',
            'TRADED_QTY':'Volume'
        })

    closing_prices = closing_prices.tail(489)
    return closing_prices




In [3]:
def moving_average(data, window_size):
    moving_averages = []
    for i in range(len(data)):
        if i < window_size - 1:
            moving_averages.append(None)  # Not enough data points to calculate the average
        else:
            window = data[i - window_size + 1:i + 1]
            moving_averages.append(sum(window) / window_size)

    up_or_down=[]
    for i in range(len(moving_averages)):
        if type(moving_averages[i])==float:
            if moving_averages[i]<data[i]:
                up_or_down.append(1)
            else:
                up_or_down.append(0)
        else:
            up_or_down.append('NA')

    return moving_averages, up_or_down



In [4]:
import warnings

warnings.filterwarnings("ignore")

def calculate_atr(high, low, close, period=14):
    df = pd.DataFrame({
        'High': high,
        'Low': low,
        'Close': close
    })
    df['Previous Close'] = df['Close'].shift(1)
    df['High-Low'] = df['High'] - df['Low']
    df['High-Previous Close'] = abs(df['High'] - df['Previous Close'])
    df['Low-Previous Close'] = abs(df['Low'] - df['Previous Close'])
    df['True Range'] = df[['High-Low', 'High-Previous Close', 'Low-Previous Close']].max(axis=1)
    df['ATR'] = df['True Range'].rolling(window=period).mean()
    return df['ATR']

def calculate_super_trend(high, low, close, atr, multiplier=3):
    df = pd.DataFrame({
        'High': high,
        'Low': low,
        'Close': close,
        'ATR': atr
    })
    df['Basic Upper Band'] = (df['High'] + df['Low']) / 2 + (multiplier * df['ATR'])
    df['Basic Lower Band'] = (df['High'] + df['Low']) / 2 - (multiplier * df['ATR'])

    df['Super Trend'] = 0.0
    df['Trend'] = 1  # 1 for Uptrend, -1 for Downtrend

    for i in range(1, len(df)):
        if df['Close'][i] > df['Super Trend'][i-1]:
            df['Super Trend'][i] = max(df['Basic Lower Band'][i], df['Super Trend'][i-1])
            df['Trend'][i] = 1
        else:
            df['Super Trend'][i] = min(df['Basic Upper Band'][i], df['Super Trend'][i-1])
            df['Trend'][i] = -1

    super_trend=df['Super Trend'].tolist()
    up_or_down=[]
    for i in range(len(super_trend)):
        if type(super_trend[i])==float:
            if super_trend[i]<close[i]:
                up_or_down.append(1)
            else:
                up_or_down.append(0)
        else:
            up_or_down.append('NA')

    return up_or_down



In [35]:


warnings.filterwarnings("ignore")

def calculate_atr_weekly(high, low, close, period=14, chunk_size=5):
    data = {
        'High': high,
        'Low': low,
        'Close': close
    }
    df = pd.DataFrame(data)

    # Calculate True Range
    df['Previous Close'] = df['Close'].shift(1)
    df['High-Low'] = df['High'] - df['Low']
    df['High-Previous Close'] = abs(df['High'] - df['Previous Close'])
    df['Low-Previous Close'] = abs(df['Low'] - df['Previous Close'])
    df['True Range'] = df[['High-Low', 'High-Previous Close', 'Low-Previous Close']].max(axis=1)

    # Group by chunks of `chunk_size`
    chunked = df.groupby(df.index // chunk_size).agg({
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'True Range': 'mean'
    })

    # Calculate ATR on chunked data
    chunked['ATR'] = chunked['True Range'].rolling(window=period).mean()
    return chunked

def calculate_super_trend_weekly(high, low, close, period=14, multiplier=3, chunk_size=5):
    df_chunked = calculate_atr_chunked(high, low, close, period, chunk_size)
    
    df_chunked['Basic Upper Band'] = (df_chunked['High'] + df_chunked['Low']) / 2 + (multiplier * df_chunked['ATR'])
    df_chunked['Basic Lower Band'] = (df_chunked['High'] + df_chunked['Low']) / 2 - (multiplier * df_chunked['ATR'])

    df_chunked['Super Trend'] = 0.0
    df_chunked['Trend'] = 1  # 1 for Uptrend, -1 for Downtrend

    for i in range(1, len(df_chunked)):
        if df_chunked['Close'][i] > df_chunked['Super Trend'][i-1]:
            df_chunked['Super Trend'][i] = max(df_chunked['Basic Lower Band'][i], df_chunked['Super Trend'][i-1])
            df_chunked['Trend'][i] = 1
        else:
            df_chunked['Super Trend'][i] = min(df_chunked['Basic Upper Band'][i], df_chunked['Super Trend'][i-1])
            df_chunked['Trend'][i] = -1

    super_trend = df_chunked['Super Trend'].tolist()
    up_or_down = []
    for i in range(len(super_trend)):
        if type(super_trend[i]) == float:
            if super_trend[i] < df_chunked['Close'].iloc[i]:
                up_or_down.append(1)
            else:
                up_or_down.append(0)
        else:
            up_or_down.append('NA')

    return df_chunked, up_or_down

# Example usage:
# Assuming you have lists `high`, `low`, and `close` for the stock data
# df_chunked, trend = calculate_super_trend_chunked(high, low, close)


In [5]:
def calculate_rsi(prices, period=14):

    prices = pd.Series(prices)
    delta = prices.diff(1)

    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    rsi_values=rsi.tolist()

    up_or_down=[]
    for i in range(len(rsi_values)):
        if type(rsi_values[i])==float:
            if rsi_values[i]>=80:
                up_or_down.append(1)
            else:
                up_or_down.append(0)
        else:
            up_or_down.append('NA')

    return rsi_values, up_or_down



In [6]:
def calculate_donchian_channel(stock_prices, high_prices, low_prices, period=20):

    high_prices = pd.Series(high_prices)
    low_prices = pd.Series(low_prices)

    upper_band = high_prices.rolling(window=period).max().tolist()
    lower_band = low_prices.rolling(window=period).min().tolist()

    up_or_down=[]
    for i in range(1,len(upper_band)):
        if type(upper_band[i])==float:
            if upper_band[i-1]<=stock_prices[i]:
                up_or_down.append(1)
            else:
                up_or_down.append(0)
        else:
            up_or_down.append('NA')

    upswing = up_or_down

    up_or_down=[]
    for i in range(1,len(lower_band)):
        if type(lower_band[i])==float:
            if lower_band[i-1]>=stock_prices[i]:
                up_or_down.append(1)

            else:
                up_or_down.append(0)
        else:
            up_or_down.append('NA')

    breakdown=up_or_down

    return upswing, breakdown

In [7]:
def calculate_donchian_ratios(closing_prices, period=20):

    high_prices = pd.Series(closing_prices)
    upper_band = high_prices.rolling(window=period).max()
    closing_band= upper_band.tolist()

    up_or_down=[]
    for i in range(1,len(closing_band)):
        if type(closing_band[i-1])==float:
            if closing_band[i-1]<closing_prices[i]:
                up_or_down.append(1)
            else:
                up_or_down.append(0)
        else:
            up_or_down.append('NA')

    return up_or_down


In [8]:

def calculate_bollinger_bands(prices, period=20, std_dev=2):

    prices = pd.Series(prices)

    middle_band = prices.rolling(window=period).mean()
    rolling_std = prices.rolling(window=period).std()

    upper_band = middle_band + (rolling_std * std_dev)
    lower_band = middle_band - (rolling_std * std_dev)

    # Bandwidth calculation
    bandwidth = ((upper_band - lower_band) / middle_band).tolist()

    expanding=[]
    for i in range(5, len(bandwidth)):
        if bandwidth[i]>bandwidth[i-5]:
            expanding.append(1)
        else:
            expanding.append(0)

    return expanding




# Steps in Analysis

### 1. check moving average: If stock has been above 20, 50, 200 MA for past 20 days
### 2. check super trend: If stock has been above super trend for the past 20 days (ASK WHICH PERIOD)
### 3. check delivery based buying: volume breakout and price breakout (ASK HOW MUCH PERCENT, AND HOW MANY DAYS)
### 4. check donchian channel
### 5. check rsi

In [54]:
NIFTY = get_stock_data('^NSEI')
NIFTY = list(NIFTY['Close'])

def check_values(name):
    # Get stock data
    stock_price_data = get_stock_data(name)
    stock_prices = list(stock_price_data['Close'])

    # Normalize stock prices
    normalised_stock_prices = [stock_prices[i] / NIFTY[i] for i in range(len(stock_prices))]
    normalised_stock_prices = [i / normalised_stock_prices[0] for i in normalised_stock_prices]

    high_prices = list(stock_price_data['High'])
    low_prices = list(stock_price_data['Low'])

    # Check moving average
    stock_moving_avg = moving_average(stock_prices, 50)[1][-50:]
    stock_moving_avg_last = stock_moving_avg[-1]
    moving_avg = moving_average(normalised_stock_prices, 50)[1][-50:]
    moving_avg_last = moving_avg[-1]
    try:
        stock_moving_avg_last_zero = (stock_moving_avg[::-1]).index(0)
    except:
        stock_moving_avg_last_zero = 'NA'
    
    try:
        moving_avg_last_zero = (moving_avg[::-1]).index(0)
    except:
        moving_avg_last_zero = 'NA'

    # Check Super Trend
    atr = calculate_atr(high_prices, low_prices, stock_prices)
    super_trend = calculate_super_trend( high_prices, low_prices, stock_prices, atr)
    super_trend = super_trend[-50:]

    if 0 in super_trend:
        last_neg = super_trend[::-1].index(0)
    else:
        last_neg = "NA"
    super_trend_last = super_trend[-1]
    
    # Check Weekly Super Trend
    weekly_atr = calculate_atr_weekly(high_prices, low_prices, stock_prices)
    super_trend_weekly = calculate_super_trend_weekly(high_prices, low_prices, stock_prices)[-1]
    super_trend_weekly = super_trend_weekly[-15:]

    if 0 in super_trend_weekly:
        last_neg_weekly = super_trend_weekly[::-1].index(0)
    else:
        last_neg_weekly = "NA"
    super_trend_last_weekly = super_trend_weekly[-1]

    # Check stock volume breakout
    stock_volume = list(stock_price_data['Volume'])
    stock_volume_MA = moving_average(stock_volume, 50)[0]

    up_or_down = []
    for i in range(len(stock_volume_MA)):
        if type(stock_volume_MA[i]) == float:
            if 2 * stock_volume_MA[i] < stock_volume[i]:
                up_or_down.append(1)
            else:
                up_or_down.append(0)
        else:
            up_or_down.append('NA')

    volume_breakout = np.average(up_or_down[-10:]) # Check when do you want the last volume breakout

    # Check upswing, breakdown on Donchian Channel
    upswing, breakdown = calculate_donchian_channel(stock_prices, high_prices, low_prices, period=20)
    upswing_avg = (np.average(upswing[-5:])>0)
    breakdown_avg = (np.average(breakdown[-5:])>0)


    # Check Donchian for ratio
    donchian_ratios = calculate_donchian_ratios(normalised_stock_prices, period=20)
    donchian_ratios_avg = np.average(donchian_ratios[-5:])

    # Check RSI
    rsi_values = calculate_rsi(normalised_stock_prices, period=14)
    rsi_latest = rsi_values[0][-1]
    rsi_values_stock = calculate_rsi(stock_prices, period=14)
    rsi_stock_latest = rsi_values_stock[0][-1]

    # Check if higher high being made
    n=normalised_stock_prices[-5:]
    n1=n.copy()
    n1.sort()
    higher_high = (n==n1)

    rsi_up = rsi_values[0][-1] > rsi_values[0][-3]

    # check if stock is positive today
    positive = (stock_prices[-1] > stock_prices[-2])

    # Check if Bollinger is expanding
    expanding = calculate_bollinger_bands(stock_prices, period=20, std_dev=2)
    bollinger_expanding = expanding[-1]

    # Create a dictionary for the results
    results = {
        'Stock Name':[name],
        'Above Moving Average': [stock_moving_avg_last],
        'Moving Average last Negative': [stock_moving_avg_last_zero],
        'Ratio Above Moving Average': [moving_avg_last],
        'Ratio Moving Average last Negative': [moving_avg_last_zero],
        'Positive Super Trend': [super_trend_last],
        'Last Negative Super Trend': [last_neg],
        'Positive Weekly Super Trend': [super_trend_last_weekly],
        'Last Negative Week Super Trend': [last_neg_weekly],
        'Volume Breakout in Last 10 Days': [volume_breakout],
        'Donchian Upswing in Last 5 Days': [upswing_avg*1],
        'Donchian Breakdown in Last 5 Days': [breakdown_avg*1],
        'Donchian Upswing Today' : [upswing[-1]],
        'Donchian Breakdown Today' : [breakdown[-1]],
        'Ratio Donchian Upswing Last 5 Days': [donchian_ratios_avg],
        'Ratio Latest RSI': [rsi_latest],
        'Latest RSI': [rsi_stock_latest],
        'Higher High in Last 2 Days': [higher_high*1],
        'RSI Going Up': [rsi_up*1],
        'Bollinger Bands Expanding': [bollinger_expanding],
        'Positive Today': [positive*1]
    }

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(results)

    return df

# Example usage
df_results = check_values('NIFTY BANK')


In [55]:
stock_list = pd.read_csv('top_1000_stocks.csv')
stock_list = list(stock_list['Symbol'])[:10]

In [56]:
all_results = pd.DataFrame()

# Iterate over the list of stocks and collect results
for stock in tqdm(stock_list):
    try:
        df_results = check_values(stock)
        all_results = pd.concat([all_results, df_results], ignore_index=True)
    except:
        continue
all_results

100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  5.34it/s]


,Stock Name,Above Moving Average,Moving Average last Negative,Ratio Above Moving Average,Ratio Moving Average last Negative,Positive Super Trend,Last Negative Super Trend,Positive Weekly Super Trend,Last Negative Week Super Trend,Volume Breakout in Last 10 Days,...,Donchian Breakdown in Last 5 Days,Donchian Upswing Today,Donchian Breakdown Today,Ratio Donchian Upswing Last 5 Days,Ratio Latest RSI,Latest RSI,Higher High in Last 2 Days,RSI Going Up,Bollinger Bands Expanding,Positive Today
0,RELIANCE.BO,0,0,0,0,1,4,1,2,0.2,...,0,0,0,0.0,41.174205,51.661383,0,0,0,1
1,TCS.BO,1,41,1,7,1,40,1,9,0.0,...,0,0,0,0.4,53.598169,62.201696,0,0,1,0
2,HDFCBANK.BO,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0.0,36.186526,42.737956,0,0,0,0
3,ICICIBANK.BO,1,2,0,0,1,6,1,NA,0.0,...,0,0,0,0.0,45.787318,55.828872,0,1,0,1
4,BHARTIARTL.BO,1,NA,0,0,1,NA,1,NA,0.1,...,0,0,0,0.0,45.566190,55.136295,0,1,1,1
5,SBIN.BO,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0.0,34.291917,38.547873,0,1,0,0
6,INFY.BO,1,NA,1,NA,1,10,1,2,0.0,...,0,0,0,0.0,48.647305,56.330184,0,1,1,0
7,LICI.BO,0,0,0,0,1,5,0,0,0.0,...,0,0,0,0.0,29.673248,32.900485,0,0,1,0
8,HINDUNILVR.BO,1,NA,1,4,1,NA,1,NA,0.0,...,0,1,0,0.0,57.793672,78.209860,0,1,1,1
9,ITC.BO,1,36,1,32,1,NA,1,14,0.0,...,0,0,0,0.0,51.804410,68.141664,0,1,0,1


In [57]:
all_results.to_excel('stock_analysis_results.xlsx', index=False)

In [58]:
index_names = [
    "NIFTY AUTO",
    "NIFTY BANK",
    "NIFTY FINANCIAL SERVICES",
    "NIFTY FINANCIAL SERVICES 25/50",
    "NIFTY FINANCIAL SERVICES EX-BANK",
    "NIFTY FMCG",
    "NIFTY HEALTHCARE",
    "NIFTY IT",
    "NIFTY MEDIA",
    "NIFTY METAL",
    "NIFTY PHARMA",
    "NIFTY PRIVATE BANK",
    "NIFTY PSU BANK",
    "NIFTY REALTY",
    "NIFTY CONSUMER DURABLES",
    "NIFTY OIL & GAS",
    "NIFTY MIDSMALL FINANCIAL SERVICES",
    "NIFTY MIDSMALL HEALTHCARE",
    "NIFTY MIDSMALL IT & TELECOM",
    "NIFTY 50",
    "NIFTY NEXT 50",
    "NIFTY 100",
    "NIFTY 200",
    "NIFTY TOTAL MARKET",
    "NIFTY 500",
    "NIFTY 500 MULTICAP 50:25:25",
    "NIFTY 500 LARGEMIDSMALL EQUAL-CAP WEIGHTED",
    "NIFTY MIDCAP 150",
    "NIFTY MIDCAP 50",
    "NIFTY MIDCAP SELECT",
    "NIFTY MIDCAP 100",
    "NIFTY SMALLCAP 250",
    "NIFTY SMALLCAP 50",
    "NIFTY SMALLCAP 100",
    "NIFTY MICROCAP 250",
    "NIFTY LARGEMIDCAP 250",
    "NIFTY MIDSMALLCAP 400",
    "INDIA VIX",
    "NIFTY COMMODITIES",
    "NIFTY CORE HOUSING",
    "NIFTY CPSE",
    "NIFTY EV & NEW AGE AUTOMOTIVE",
    "NIFTY ENERGY",
    "NIFTY HOUSING",
    "NIFTY INDIA CONSUMPTION",
    "NIFTY INDIA DEFENCE",
    "NIFTY INDIA DIGITAL",
    "NIFTY INDIA MANUFACTURING",
    "NIFTY INDIA TOURISM",
    "NIFTY INFRASTRUCTURE",
    "NIFTY IPO",
    "NIFTY MIDCAP LIQUID 15",
    "NIFTY MIDSMALL INDIA CONSUMPTION",
    "NIFTY MNC",
    "NIFTY MOBILITY",
    "NIFTY PSE",
    "NIFTY REITS & INVITS",
    "NIFTY NON-CYCLICAL CONSUMER",
    "NIFTY SERVICES SECTOR",
    "NIFTY SHARIAH 25",
    "NIFTY100 LIQUID 15",
    "NIFTY50 SHARIAH",
    "NIFTY500 SHARIAH",
    "NIFTY500 MULTICAP INDIA MANUFACTURING 50:30:20",
    "NIFTY500 MULTICAP INFRASTRUCTURE 50:30:20",
    "NIFTY SME EMERGE",
    "NIFTY TRANSPORTATION & LOGISTICS",
    "NIFTY100 ESG",
    "NIFTY100 ENHANCED ESG",
    "NIFTY100 ESG SECTOR LEADERS"
]


In [14]:

all_results = pd.DataFrame()

# Iterate over the list of stocks and collect results
for stock in tqdm(index_names):
    try:
        df_results = check_values(stock)
        all_results = pd.concat([all_results, df_results], ignore_index=True)
    except:
        continue
all_results

100%|███████████████████████████████████████████| 70/70 [02:50<00:00,  2.43s/it]


,Stock Name,Ratio Above Moving Average,Ratio Moving Average last Negative,Positive Super Trend,Last Negative Super Trend,Volume Breakout in Last 10 Days,Donchian Upswing in Last 5 Days,Donchian Breakdown in Last 5 Days,Donchian Upswing Today,Donchian Breakdown Today,Ratio Donchian Upswing Last 5 Days,Ratio Latest RSI,Latest RSI,Higher High in Last 2 Days,RSI Going Up,Bollinger Bands Expanding,Positive Today
0,NIFTY AUTO,0,0,1,4,0.0,0,0,0,0,0.0,60.440218,65.136238,0,1,0,0
1,NIFTY BANK,0,0,1,2,0.0,0,0,0,0,0.0,56.212902,61.712278,0,1,0,1
2,NIFTY FINANCIAL SERVICES,0,0,1,4,0.0,0,0,0,0,0.0,57.635167,61.995196,0,1,0,1
3,NIFTY FINANCIAL SERVICES 25/50,0,0,1,6,0.0,0,0,0,0,0.0,60.739126,64.989998,0,1,1,1
4,NIFTY FMCG,1,31,1,NA,0.0,1,0,0,0,0.0,60.632413,72.126776,0,1,1,1
5,NIFTY IT,1,40,1,11,0.0,1,0,0,0,0.2,69.409855,77.764213,0,1,1,1
6,NIFTY MEDIA,0,0,1,23,0.0,0,0,0,0,0.0,59.407975,63.758993,0,1,0,1
7,NIFTY METAL,0,0,0,0,0.0,0,0,0,0,0.0,63.530835,67.753914,0,1,0,1
8,NIFTY PHARMA,1,25,1,NA,0.0,1,0,0,0,0.0,65.232520,82.147745,0,0,0,1
9,NIFTY PRIVATE BANK,0,0,1,4,0.0,0,0,0,0,0.0,57.696173,63.421207,0,1,0,1


In [15]:
all_results.to_excel('index_analysis_results.xlsx', index=False)

In [16]:
k=capital_market.market_watch_all_indices()
adr=int(k['declines'][4])/int(k['advances'][4])
vix=list(capital_market.india_vix_data(period='1W')['CLOSE_INDEX_VAL'])[-1]

import openpyxl
from datetime import datetime

# Function to append new data to the next empty row in the Excel sheet
def append_data_to_excel(file_path, date, adr_value, vix_value):
    # Load the workbook
    workbook = openpyxl.load_workbook(file_path)

    # Select the sheet you want to work with
    sheet = workbook['Sheet1']  # Change 'Sheet1' to the name of your sheet

    # Find the first empty row
    next_row = sheet.max_row + 1
    for row in range(1, sheet.max_row + 1):
        if sheet[f'A{row}'].value is None:
            next_row = row
            break

    # Read the last date in the sheet
    last_date_cell = sheet[f'A{next_row - 1}']
    if last_date_cell.value is None:
        last_date_cell = sheet[f'A{next_row - 2}']
    last_date = last_date_cell.value

    # Check if the date is different from today's date
    if last_date != date:
        # Append the new data
        sheet[f'A{next_row}'] = date
        sheet[f'B{next_row}'] = adr_value
        sheet[f'C{next_row}'] = vix_value

        # Save the workbook with the changes
        workbook.save(file_path)
        print("Data appended successfully.")
    else:
        print("The date is the same as the last date in the sheet. No data appended.")

# Example usage
file_path = 'ADRvsVIX.xlsx'
today_date = datetime.today().strftime('%d-%b-%Y')

append_data_to_excel(file_path, today_date, adr, vix)


Data appended successfully.
